## Data prep and loading

### Data config and result naming

In [1]:
# reads_dataset_name = "hsa.dRNASeq.HeLa.polyA.REL5.long.1"
# reads_dataset_name = "hsa.dRNASeq.HeLa.polyA.REL5.long.2"
reads_dataset_name = "ds2to11"

reads_dataset_preprocessing = 'cds_local.max.ds2to11'
# reads_dataset_preprocessing = 'original'
addition = "adaptive"

SEQ_LENGTH = 12 # after processing it will actually be == SEQ_LENGTH + 1 

folder_run_id = 'rad5' + reads_dataset_preprocessing + "_" + addition + "_wd0.01_" + str(SEQ_LENGTH + 1) + 'n'

dataset_save_name = 'rad5' + reads_dataset_preprocessing + "_" + addition
if SEQ_LENGTH != 200:
    dataset_save_name = dataset_save_name + str(SEQ_LENGTH) + 'n'
dataset_save_name

'rad5cds_local.max.ds2to11_adaptive12n'

In [2]:
import comet_ml
from torch import cuda
cuda.is_available()

/home/jovyan/my-conda-envs/decay2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
import os

os.environ['COMET_API_KEY'] = "EpKIINrla6U4B4LJhd9Sv4i0b"

# Commet Init
comet_ml.init(project_name="decay", api_key= "EpKIINrla6U4B4LJhd9Sv4i0b")


COMET INFO: Comet API key is valid
COMET INFO: Comet API key saved in /home/jovyan/.comet.config


### Func to load data

In [4]:
import sys
import argparse
import datetime
import numpy as np
from Bio import SeqIO
from math import ceil

from sklearn.model_selection import train_test_split

from raw_to_input import *
from raw_to_input import reformat, parse_data, parse_neg


def load_data(upstream5p, downstream3p, within, rbpfile, negfile, rep, onehot):
    """Parses and processes data from input FASTA and BED files and returns it in train and testing portions.
    @param upstream5p
    """
    # I do not use the RBPs now, but leaving the code here for later
    rbp_dict = get_rbp_dict(rbpfile)

    x, rbps, y = [], [], []
    # Iterate through files and append sequences and corresponding labels and region information to above data arrays
    parse_data(upstream5p, x, y, rbps, rbp_dict, 5, rep, onehot, length=SEQ_LENGTH)
    print(len(y), ' processed upstream5p; ', 'currect label balance: ', y.count(1), ' ones, ', y.count(0), ' zeros')
    parse_data(downstream3p, x, y, rbps, rbp_dict, 3, rep, onehot, length=SEQ_LENGTH)
    print(len(y), ' processed downstream3p; ', 'currect label balance: ', y.count(1), ' ones, ', y.count(0), ' zeros')
    parse_data(within, x, y, rbps, rbp_dict, "w", rep, onehot, length=SEQ_LENGTH)
    print(len(y), ' processed within; ', 'currect label balance: ', y.count(1), ' ones, ', y.count(0), ' zeros')

    x_neg, rbps_neg, y_neg = [], [], []
    
    # generate negatives
    # in each cycle there we add 1 random positive
    while y.count(1) > y_neg.count(0):
        parse_neg(negfile, x_neg, y_neg, rbps_neg, rbp_dict, rep, onehot, length=SEQ_LENGTH)
        print(len(y_neg), ' processed negfile; ', 'current label balance: ', y.count(1), ' ones, ', y_neg.count(0), ' zeros')
    
    # combine the positives with the generated negatives
    x.extend(x_neg[:len(y)])    
    rbps.extend(rbps_neg[:len(y)])  
    y.extend(y_neg[:len(y)])
    print(len(y), ' total; ', 'label balance: ', y.count(1), ' ones, ', y.count(0), ' zeros')
    
    return x, rbps, y

2023-11-09 01:30:11.585036: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Set data paths

In [5]:
home_path = '/home/jovyan'
pos_5p_train = home_path + '/decay/decay/get_fasta/results/{}/{}/decay_seqs_fasta/upstream-5P-end-transcripts_fasta_train.fa'.format(reads_dataset_name, reads_dataset_preprocessing + "/" + addition)
pos_5p_test = home_path + '/decay/decay/get_fasta/results/{}/{}/decay_seqs_fasta/upstream-5P-end-transcripts_fasta_test.fa'.format(reads_dataset_name, reads_dataset_preprocessing + "/" + addition)

pos_3p_train = home_path + '/decay/decay/get_fasta/results/{}/{}/decay_seqs_fasta/downstream-3P-end-transcripts_fasta_train.fa'.format(reads_dataset_name, reads_dataset_preprocessing + "/" + addition)
pos_3p_test = home_path + '/decay/decay/get_fasta/results/{}/{}/decay_seqs_fasta/downstream-3P-end-transcripts_fasta_test.fa'.format(reads_dataset_name, reads_dataset_preprocessing + "/" + addition)

within_train = home_path + '/decay/decay/get_fasta/results/{}/{}/decay_seqs_fasta/within_transcripts_fasta_train.fa'.format(reads_dataset_name, reads_dataset_preprocessing + "/" + addition)
within_test = home_path + '/decay/decay/get_fasta/results/{}/{}/decay_seqs_fasta/within_transcripts_fasta_test.fa'.format(reads_dataset_name, reads_dataset_preprocessing + "/" + addition)

neg_file = home_path + '/decay/decay/data/hg38/transcripts.fa'

rbp_file = home_path + '/decay/decay/rbp_data/AATF.bed'

rep = False # was True --- switch for soft-masked and hard-masked nucleotides 
onehot = False # Return nucleotide characters or one-hot vectors


In [6]:
pos_5p_train

'/home/jovyan/decay/decay/get_fasta/results/ds2to11/cds_local.max.ds2to11/adaptive/decay_seqs_fasta/upstream-5P-end-transcripts_fasta_train.fa'

### Load data

In [7]:
if not os.path.exists(dataset_save_name):
    # Load train data
    x_train_raw, rbps_train, y_train = load_data(
        pos_5p_train, pos_3p_train, within_train, 
        rbp_file, neg_file, rep, onehot)

    # Load test data
    x_test_raw, rbps_test, y_test = load_data(
        pos_5p_test, pos_3p_test, within_test, 
        rbp_file, neg_file, rep, onehot)



#     if onehot:
#         n_seqlength, n_features = x_train_raw[0].shape[0], x_train_raw[0].shape[1]
#     else:
#         n_seqlength, n_features = np.shape(x_train_raw)[0], np.shape(x_train_raw)[1]

    # np.shape(x_train_raw), np.shape(x_test_raw), np.shape(rbps_train), np.shape(rbps_test), np.shape(y_train), np.shape(y_test)

In [8]:
if not os.path.exists(dataset_save_name):
    print(np.shape(y_train), np.shape(y_test))
    print(np.shape(x_train_raw), np.shape(x_test_raw))

In [9]:
# for i in range(50):
#     print(len(x_test_raw[i*(-1)]))

# Model and training

### Config

In [10]:
### Parameters
#### Mostly unchanged
RANDOMIZE_WEIGHTS = False 

MODEL_NAME = "simecek/DNADebertaK6c"
TOKENIZER_NAME = "armheb/DNA_bert_6"
K = 6
STRIDE = 1

# Lower batch_size and increase numbre of accumulation steps if your machine strugles with memory
BATCH_SIZE = 64
# Higher accumulation could help with overfitting by "decreasing the number of update steps"
ACCUMULATION = 1

# set
EPOCHS = 1
# EPOCHS = 5

#### Mostly unchanged
LEARNING_RATE = 1e-5
RUNS = 1
EARLY_STOPPING_PATIENCE = 3
USE_CLASS_WEIGHTS = False
hard_nucleotide_masking = False

SEED=42
np.random.seed(SEED)

# warmup_ratio = 0.05 #5 epochs (for 100 epochs total train)
warmup_ratio = 0.20
if(RANDOMIZE_WEIGHTS):
    warmup_ratio = 0
    


In [11]:
folder_run_id

'rad5cds_local.max.ds2to11_adaptive_wd0.01_13n'

### Training args

In [12]:
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback
from transformers.integrations import CometCallback


def get_trainargs():
    return TrainingArguments(
        'outputs' + '/' + folder_run_id, 
        learning_rate=LEARNING_RATE, 
        warmup_ratio=warmup_ratio, 
        lr_scheduler_type='linear',
        fp16=True,
        per_device_train_batch_size=BATCH_SIZE, 
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=ACCUMULATION,
        num_train_epochs=EPOCHS, 
        # max_steps=4500, # there is 17 898 Total optimization steps in 1 epoch
        # save_steps=4500,     # Frequency of saving checkpoints
        # logging_steps=4500,   # Logging frequency
        # save_strategy='steps',
        # evaluation_strategy="steps", 
        weight_decay=0.01,
        evaluation_strategy="epoch", 
        save_strategy='epoch',
        # save_strategy='no',
        seed=randrange(1,10001), 
        report_to='none',
        load_best_model_at_end=True,
        metric_for_best_model='eval_loss'
    )
#early stopping 5 epochs
callbacks= [
    EarlyStoppingCallback(early_stopping_patience=EARLY_STOPPING_PATIENCE, early_stopping_threshold=0.001),
    CometCallback()
]

### Get tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME, trust_remote_code=True)

### Nucleotide masking

- so far, it is either hard-masking or no-masking
- there is no soft-masking possible with this \**pretrained*\* model because the model's alphabet contains only "A, C, T, G, UNK" tokens

In [ ]:
from datasets import Dataset, DatasetDict

if not os.path.exists(dataset_save_name):
    
    "".join(x_train_raw[100])
    # x_train_raw.dtype

    "".join(x_test_raw[0])

    if not hard_nucleotide_masking:
        # hack to unmask nucleotides
        try:
            x_train = np.char.upper(x_train_raw)
        except:
            x_train = np.char.upper(x_train_raw.astype(str))

        try:
            x_test = np.char.upper(x_test_raw)
        except:
            x_test = np.char.upper(x_test_raw.astype(str))
    else:
        # no action needed, the tokenizer will map lower case chars to UNK tokens
        pass

    "".join(x_train[100])

    ### Tokenize

    def kmers_strideK(s, k=K):
        return [s[i:i + k] for i in range(0, len(s), k) if i + k <= len(s)]

    def kmers_stride1(s, k=K):
        return [s[i:i + k] for i in range(0, len(s)-k+1)]

    if (STRIDE == 1):
        kmers = kmers_stride1
    else:
        kmers = kmers_strideK

    # function used for the actual tokenization
    if(K is not None):
        def tok_func(x): return tokenizer(" ".join(kmers(x["seq"])), truncation=True)
    else:
        def tok_func(x): return tokenizer(x["seq"], truncation=True)

    # example
    example = tok_func({'seq': "".join(x_train[100])})    
    print(example)
    tokenizer.decode(example['input_ids'])

    len(example['input_ids'])

    tmp_dict_train = {}
    # TRAIN
    for index, (x, y) in enumerate(zip(x_train, y_train)):
        tmp_dict_train[f"{index}"] = ("train", y, x)

    tmp_dict_test = {}
    # TEST
    for index, (x, y) in enumerate(zip(x_test, y_test)):
        tmp_dict_test[f"{index}"] = ("test", y, x)

    df_train = pd.DataFrame.from_dict(tmp_dict_train).T.rename(columns = {0: "dset", 1: "cat", 2: "seq"})
    df_test = pd.DataFrame.from_dict(tmp_dict_test).T.rename(columns = {0: "dset", 1: "cat", 2: "seq"})

    df_train['seq'] = [''.join(map(str, l)) for l in df_train['seq']]
    df_test['seq'] = [''.join(map(str, l)) for l in df_test['seq']]


    ds_train = Dataset.from_pandas(df_train)
    ds_test = Dataset.from_pandas(df_test)

    # tok_ds = ds.map(tok_func, batched=False, remove_columns=['__index_level_0__', 'seq'])
    tok_ds_train = ds_train.map(tok_func, batched=False, remove_columns=['__index_level_0__', 'seq'])
    tok_ds_train = tok_ds_train.rename_columns({'cat':'labels'})

    tok_ds_test = ds_test.map(tok_func, batched=False, remove_columns=['__index_level_0__', 'seq'])
    tok_ds_test = tok_ds_test.rename_columns({'cat':'labels'})


    dds = DatasetDict({
        'train': tok_ds_train,
        'valid': [],
        'test': tok_ds_test
    })

    train_valid_split = dds['train'].train_test_split(test_size=0.2, shuffle=True, seed=42)
    dds['train']=train_valid_split['train']
    dds['valid']=train_valid_split['test']

### Save or load dataset

In [ ]:
from datasets import load_from_disk

if os.path.exists(dataset_save_name):
    print("File exists:", dataset_save_name)
    dds = load_from_disk(dataset_save_name)
else:
    print("File not found:", dataset_save_name)
    dds.save_to_disk(dataset_save_name)
    

In [ ]:
# def shorten_sequence(sample):
#     sample['input_ids'] = sample['input_ids']
#     return sample

# updated_dataset = dds['test'].map(shorten_sequence)

In [ ]:
# print(updated_dataset['input_ids'][0][:50])

In [ ]:
# for i in range(5):
#     print(updated_dataset['input_ids'][i][:20])

In [ ]:
# tmp = tokenizer.decode(updated_dataset['input_ids'][0])

In [ ]:
# tmp

In [ ]:
# # truncation of # of front and # of back
# def remove_lateral_tokens(seq, remove_number = 50):
#     splited_seq = seq.split()
    
#     start_tok = [splited_seq.pop(0)]
#     end_tok = [splited_seq.pop(-1)]
#     print(start_tok, end_tok, splited_seq)
#     print(len(splited_seq))
    
#     shortened = splited_seq[remove_number:][:-remove_number]
#     start_tok.extend(shortened)
#     start_tok.extend(end_tok)
#     return " ".join(start_tok)
    
# # print(remove_lateral_tokens(tmp))    
# len(tokenizer.encode(remove_lateral_tokens(tmp)))

### Logs

Function to extract dataframe metrics row from training logs

In [ ]:
def get_log_from_history(history, dataset_name):
    eval_dicts = [x for x in history if 'eval_loss' in x]
    test_dicts = [x for x in history if 'test_loss' in x]
    test_log = test_dicts[0]
    test_acc = test_log['test_accuracy']
    test_f1 = test_log['test_f1']
    test_loss = test_log['test_loss']
    test_precision = test_log['test_precision']
    test_recall = test_log['test_recall']
    test_auroc_macro = test_log['test_rocauc_0_roc_auc']
    test_auroc_weighted = test_log['test_rocauc_1_roc_auc']
    test_pr_auc = test_log['test_pr_auc']
    
    
    min_loss_dict = min(eval_dicts, key=lambda x: x['eval_loss'])
    min_loss_epoch = min_loss_dict['epoch']
    # max_f1_dict = max(eval_dicts, key=lambda x: x['eval_f1'])
    # max_acc_dict = max(eval_dicts, key=lambda x: x['eval_accuracy'])
    row = {
        'dataset':dataset_name,
        'test_acc':test_acc,
        'test_f1':test_f1,
        'test_loss':test_loss,
        'test_precision':test_precision,
        'test_recall':test_recall,
        'test_auroc_macro':test_auroc_macro,
        'test_auroc_weighted':test_auroc_weighted,
        'test_pr_auc':test_pr_auc,
        
        'min_valid_loss_epoch':min_loss_epoch,
        'min_valid_loss_log':min_loss_dict,
        # 'max_valid_f1_log':max_f1_dict,
        # 'max_valid_acc_log':max_acc_dict,
    }
    return row

In [ ]:
import evaluate
binary_metrics = evaluate.combine([
    'accuracy',
    'f1',
    'recall',
    'precision',
    #Order of roc_auc matters for logging -> macro first, then weighted
    evaluate.load('roc_auc', average='macro'),
    evaluate.load('roc_auc', average='weighted'),
    evaluate.load("Vlasta/pr_auc"),
])


In [ ]:
import pandas as pd
from random import random, randrange
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import torch

def compute_metrics_binary(eval_preds):
    logits, labels = eval_preds
    prediction_scores = torch.nn.functional.softmax(
        torch.from_numpy(logits).double(), dim=-1).numpy() 
    # predictions = np.argmax(logits, axis=-1) #equivalent
    predictions = np.argmax(prediction_scores, axis=-1)
    return binary_metrics.compute(
        predictions=predictions, 
        references=labels, 
        prediction_scores=prediction_scores[:,1] #taking only prediction percentage for the label 1
    )



### Training

In [ ]:
# Custom trainer for class weights in compute_loss
from torch import nn, save

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([11.68802694, 1.0]).to('cuda'))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1)).to('cuda')
        return (loss, outputs) if return_outputs else loss


In [ ]:
dataset_name="decay"
compute_metrics = compute_metrics_binary
outputs = []

if(USE_CLASS_WEIGHTS):
    Trainer_to_use = CustomTrainer
else:
    Trainer_to_use = Trainer

for _ in range(RUNS):
    model_cls = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
    if(RANDOMIZE_WEIGHTS):
        # model_cls.init_weights() #Alternative
        model_cls = AutoModelForSequenceClassification.from_config(model_cls.config)            

    args = get_trainargs()
    
    trainer = Trainer_to_use(model_cls, args, train_dataset=dds['train'], eval_dataset=dds['valid'],
                      tokenizer=tokenizer, compute_metrics=compute_metrics, 
                      callbacks=callbacks) 
        
    trainer.train()
    # trainer.evaluate(dds['test'], metric_key_prefix='test')
    predictions = trainer.evaluate(dds['test'], metric_key_prefix='test')
    training_log = get_log_from_history(trainer.state.log_history, dataset_name=dataset_name)
    outputs.append(training_log)


In [ ]:
predictions

In [28]:
from sklearn import metrics

predictions = trainer.predict(dds["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

preds = np.argmax(predictions.predictions, axis=-1)

confusion_matrix_test = metrics.confusion_matrix(predictions.label_ids, preds, labels=[0, 1])

The following columns in the test set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: dset. If dset are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 163476
  Batch size = 64


(163476, 2) (163476,)


count of true negatives is C(0,0), false negatives is C(1,0), true positives is C(1,1) and false positives is C(0,1)

In [29]:
from sklearn import metrics


In [30]:
tn, fp, fn, tp = confusion_matrix_test.ravel()
print(tp + fn, tn + fp)

precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)

print('test')
print('precision ', precision)
print('recall ', recall)
print('specificity ', specificity)

confusion_matrix_test = confusion_matrix_test.tolist()
confusion_matrix_test.append({'precision': precision, 'recall': recall, 'specificity': specificity})

81738 81738
test
precision  0.6370300893733087
recall  0.7229073380802075
specificity  0.5880985588098558


In [31]:
outputs.append({'confusion_matrix_test' : confusion_matrix_test})

In [32]:
outputs

[{'dataset': 'decay',
  'test_acc': 0.6555029484450317,
  'test_f1': 0.6772572280008023,
  'test_loss': 0.622995913028717,
  'test_precision': 0.6370300893733087,
  'test_recall': 0.7229073380802075,
  'test_auroc_macro': 0.7104757603468905,
  'test_auroc_weighted': 0.7104757603468905,
  'test_pr_auc': 0.6831714813764451,
  'min_valid_loss_epoch': 1.0,
  'min_valid_loss_log': {'eval_loss': 0.6176930665969849,
   'eval_accuracy': 0.6597058289449791,
   'eval_f1': 0.6827823278948945,
   'eval_recall': 0.7320996013989431,
   'eval_precision': 0.6396901399859709,
   'eval_rocauc_0_roc_auc': 0.7176280425958544,
   'eval_rocauc_1_roc_auc': 0.7176280425958544,
   'eval_pr_auc': 0.6923296338922672,
   'eval_runtime': 46.899,
   'eval_samples_per_second': 6105.97,
   'eval_steps_per_second': 95.418,
   'epoch': 1.0,
   'step': 17898}},
 {'confusion_matrix_test': [[48070, 33668],
   [22649, 59089],
   {'precision': 0.6370300893733087,
    'recall': 0.7229073380802075,
    'specificity': 0.588098

In [33]:
import json
with open(folder_run_id + '_log.txt', 'w+') as f:
    f.write(json.dumps(outputs))

In [34]:
save(model_cls, folder_run_id + '.pth')

In [35]:
folder_run_id

'rad5cds_local.max.ds2to11_adaptive_wd0.01_13n'

In [36]:
# print(model_cls)